In [1]:
import networkx as nx
from sklearn import svm
from operator import itemgetter

import networkx as nx
import numpy as np
import random
import math
import csv
import datetime

from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import normalize
import multiprocessing as mp


from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score 
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score



from sklearn.neural_network import MLPClassifier
from sklearn import linear_model

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from zipfile import ZipFile as zipfile

In [2]:
# # importing required modules
# from zipfile import ZipFile

# # specifying the zip file name
# file_name = "facebook.zip"

# # opening the zip file in READ mode
# with ZipFile(file_name, 'r') as zip:
# 	# printing all the contents of the zip file
# 	zip.printdir()

# 	# extracting all the files
# 	print('Extracting all the files now...')
# 	zip.extractall()
# 	print('Done!')

# Building Complete Dataset Graph

In [3]:
G = nx.Graph()

In [4]:
def buildGraph(G):
    fileName="/content/facebook_combined/facebook_combined.txt"
    f=open(fileName)
    line=f.readline()
    while(line!=''):
      c=(line.split())
      G.add_edge(int(c[0]), int(c[1]))
      line=f.readline()
    f.close()
buildGraph(G)

In [5]:
def CommonNeighbors(u, v, g):
    u_n = set(g.neighbors(u))
    v_n = set(g.neighbors(v))
    return len(u_n.intersection(v_n))
def common_neighbors(g, edges):
    res = []
    for edge in edges:
        node1, node2 = edge[0], edge[1]
        num_common_neighbors = 0
        try:
            n_1, n_2 = g.neighbors(node1), g.neighbors(node2)
            for neighbor in n_1:
                if neighbor in n_2:
                    num_common_neighbors += 1
            res.append((node1, node2, num_common_neighbors))
        except:
            pass
    return res

In [6]:
featuresSet = [common_neighbors,
                   nx.resource_allocation_index,
                   nx.jaccard_coefficient,
                   nx.adamic_adar_index,
                   nx.preferential_attachment
                   ]

In [7]:
def ProduceFakeEdge(g, negative_graph,NumTestEdges):
    i = 0
    while i < NumTestEdges:
        edge = random.sample(g.nodes(), 2)
        try:
            shortestPath = nx.shortest_path_length(g,source=edge[0],target=edge[1])
            if shortestPath >= 2:
                negative_graph.add_edge(edge[0],edge[1], positive="False")
                i += 1
        except:
            pass

In [8]:
def CreateGraphFromFile(f_name):
    f = open(f_name, "rb")
    g = nx.read_edgelist(f)
    return g

In [9]:
def SampleExtraction(g, positive_number, negative_number, negative_mode, negative_distance=2, d=1):
    """

    :param g:  the graph
    :param positive_number: the number of positive samples
    :param negative_number: the number of negative samples
    :param negative_distance: the distance between two nodes in negative samples
    :param d: if d ==0, don't d positive edges from graph
    :return: positive_sample is a list of positive edges, negative_sample is a list of negative edges
    """

    # Randomly selecting positive_number as test edges
    
    positive_sample = random.sample(g.edges(), positive_number)
    sample_graph = nx.Graph()
    sample_graph.add_edges_from(positive_sample, positive="True")
    nx.write_edgelist(sample_graph, "sample_positive_" +str(positive_number)+ ".txt", data=['positive'])

    #   Adding non-existing edges to test

    i = 0
    negative_graph = nx.Graph()
    ProduceFakeEdge(g,negative_graph,negative_number)
    nx.write_edgelist(negative_graph, "sample_negative_" +str(negative_number)+ ".txt", data=["positive"])
    negative_sample = negative_graph.edges()
    negative_graph.add_edges_from(positive_sample,positive="True")
    nx.write_edgelist(negative_graph, "sample_combine_" +str(positive_number + negative_number)+ ".txt", data=["positive"])

    if d == 0:
        return positive_sample, negative_sample
    else:
        g.remove_edges_from(positive_sample)
        nx.write_edgelist(g, "training.txt", data=False)

        return positive_sample, negative_sample

In [10]:
def FeatureExtraction(g, positive_sample, negative_sample, feature_name, model="single", combine_num=5):

    data = []
    if model == "single":
        preds = feature_name(g, positive_sample)
        feature = [feature_name.__name__] + [i[2] for i in preds]
        label = ["label"] + ["Pos" for i in range(len(feature))]
        preds = feature_name(g, negative_sample)
        feature1 = [i[2] for i in preds]
        feature = feature + feature1
        label = label + ["Neg" for i in range(len(feature1))]
        data = [feature, label]
        data = trans(data)
        WriteData(data, "features_" + model + "_" + feature_name.__name__ + ".csv")
    else:
        label = ["label"] + ["1" for i in range(len(positive_sample))] + ["0" for i in range(len(negative_sample))]
        for j in feature_name:
            preds = j(g, positive_sample)

            feature = [j.__name__] + [i[2] for i in preds]
            preds = j(g, negative_sample)
            feature = feature + [i[2] for i in preds]
            data.append(feature)

        data.append(label)
        data = trans(data)
        WriteData(data, "features_" + model + "_" + str(combine_num) + ".csv")
    return data


def WriteData(data, f_name):
    csvfile = open(f_name, "w")
    writer = csv.writer(csvfile)
    for i in data:
        writer.writerow(i)
    csvfile.close()


def trans(data):
    return [list(i) for i in zip(*data)]

In [11]:
def main(f_name="/content/facebook_combined/facebook_combined.txt", positive_number=0.1, negative_number=0.1, model="combined", combine_num=1,
         feature_name=common_neighbors, negative_mode="hard"):
    if combine_num==2:
        positive_number=0.008;
        negative_number=0.008;
    g = CreateGraphFromFile(f_name)
    num_edges = g.number_of_edges()
    positive_number = int(num_edges * positive_number)
    negative_number = int(num_edges * negative_number)
    positive_sample, negative_sample = SampleExtraction(g, positive_number, negative_number,negative_mode)
    train_data = FeatureExtraction(g, positive_sample, negative_sample, feature_name, model, combine_num)

In [12]:
fn="/content/facebook_combined/facebook_combined.txt";
cn=2;

In [13]:
#Run this line to genrate feature Set
main(f_name=fn,model="combined",combine_num=cn, feature_name=featuresSet, negative_mode="easy")

In [14]:
r=np.loadtxt(open("features_combined_"+str(cn)+".csv", "rb"), delimiter=",", skiprows=1);

In [15]:
l,b=r.shape;

In [16]:
np.random.shuffle(r);

In [17]:
t_I=int(0.8*l)
X_train=r[0:t_I,0:b-1]
Y_train=r[0:t_I,b-1]

X_test=r[t_I:l,0:b-1]
Y_test=r[t_I:l,b-1]

X_train = normalize(X_train, axis=0, norm='max')
X_test = normalize(X_test, axis=0, norm='max')

scaler = StandardScaler()  
scaler.fit(X_train)  

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)  

In [18]:
def SVM_Model(train, t_lables, test, test_labels):
    #Support Vector Machine
    model_1 = svm.SVC()
    model_1.fit(train, t_lables)
    result = model_1.predict(test)

    print ("Accuracy of SVM Model:", accuracy_score(test_labels, result))

In [19]:
## Accuracy of SVM model
SVM_Model(X_train,Y_train,X_test,Y_test)

Accuracy of SVM Model: 0.640661938534279


In [20]:
def Logistic_Model(train, t_lables, test, test_labels):
    model_1 = LogisticRegression(random_state=0, solver='lbfgs',multi_class='ovr').fit(train, t_lables)
    model_1.fit(train, t_lables)
    result=model_1.predict(test)
    print ("Accuracy of Logistics Regression Model:", accuracy_score(test_labels, result))

In [21]:
#Accuracy of Logistic Regression Model
Logistic_Model(X_train,Y_train,X_test,Y_test)

Accuracy of Logistics Regression Model: 0.6595744680851063


In [22]:
def ANN_Model(train, t_lables, test, test_labels):
    model_1 = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(15,9), random_state=1)
    model_1.fit(train, t_lables)
    result = model_1.predict(test)
    print ("ANN Model Accuracy:", accuracy_score(test_labels, result))
    

In [23]:
# Accuracy of ANN Model
ANN_Model(X_train,Y_train,X_test,Y_test)

ANN Model Accuracy: 0.6572104018912529


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
